### Introduction

When we do cloud application deployment, maybe the most important thing is containerize our application into containers, like **docker**, but when we need to scale our application, we must focus on **Kubernetes** that we could scale, auto-healing our containers as we want. I have to see that for **Kubernetes** there are many concepts about it, I will try to explain it as I could. But this tutorial just focus on the hands-on for project.

Let's first get start.

In [62]:
# fist auth this notebook
from google.colab import auth
auth.authenticate_user()

In [63]:
# config our project
! gcloud config set project cloudtutorial-279003

Updated property [core/project].


#### Make a directory

In [51]:
import os

# first let's make a new folder
try:
  folder_name = "flask_web"
  os.mkdir(folder_name)

  print("Now we are in:  ", os.path.abspath(os.curdir))
except Exception as e:
  print("When we create folder with error:{}".format(e))

Now we are in:   /content/flask_web/flask_web


In [64]:
# go to folder: 'flask_web'
os.chdir('/content/{}'.format(folder_name))

#### Main logic with app.py by Flask

In [73]:
# this I just make the simple server application
%%writefile app.py
import os
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_kube():
  return "Hello, kubernetes!"

# when deploy into cloud run with error: Cloud Run error: Container failed to start. Failed to start and then listen on the port defined by the PORT environment variable. 
# so we have to define the port we would like to listen.
# As recommended from official website, we shouldn't hardcode the port!
if __name__ == "__main__":
  # app.run()
  # I don't know why, with just app.run, I always get that error!!!

  # so here just change with host and port
  app.run(debug=True, host='0.0.0.0', port=int(os.environ.get('PORT', 8080)))

Overwriting app.py


In [178]:
# then we need to provide with requirments.txt
%%writefile requirments.txt
flask == 1.1.2
gunicorn==19.9.0

Overwriting requirments.txt


In [70]:
# # then let's create our Dockerfile
# %%writefile Dockerfile
# # which image to use, when to use ubuntu, it's too big
# # let's just use python: alpine as base image
# FROM python:3-alpine
 
# # update 
# #RUN apt-get update -y && apt-get install -y python-pip python-dev

# # copy requirements file
# # COPY ./requirments.txt /app/requirments.txt
# COPY . /app

# # define the work dir

# WORKDIR /app

# # install modules
# RUN pip install  --no-cache-dir -r requirments.txt

# # here I define a port variable for later use case in cloud run, also should listen to
# ENV PORT 8080

# # ENTRYPOINT ["python"]

# CMD ["python", "app.py"]

Overwriting Dockerfile


#### Define Dockerfile

In [217]:
# As I always get error with cloud run, so here I just want to re-create the Dockerfile
%%writefile Dockerfile
FROM python:3.7-slim

# copy local file into container
ENV APP_HOME /app
WORKDIR $APP_HOME
COPY . ./

# install dependencies
# gunicorn is a WSGI http server
RUN pip install flask gunicorn

# Run the web service on container startup. Here we use the gunicorn
# webserver, with one worker process and 8 threads.
# For environments with multiple CPU cores, increase the number of workers
# to be equal to the cores available.
# CMD exec gunicorn --bind :$PORT --workers 1 --threads 4 --timeout 0 app:app
# change according to: https://stackoverflow.com/questions/47408972/connection-refused-to-gcp-loadbalancer-in-kubernetes
CMD ["gunicorn", "app:app", "-b", "0.0.0.0:8080", "-w", "3" ]

Overwriting Dockerfile


#### Build and push image with GCP

In [218]:
# first let's build the image, and put the built image into container reposity
# so that we could get the image from remote server like docker or cloud run..
! gcloud builds submit --tag gcr.io/cloudtutorial-279003/flask-container:latest

Creating temporary tarball archive of 5 file(s) totalling 2.3 KiB before compression.
Uploading tarball of [.] to [gs://cloudtutorial-279003_cloudbuild/source/1593505180.657975-20d6920b28dd4e759ca67cf216c96337.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/cloudtutorial-279003/builds/94d97f21-9567-4a5e-a67a-2615db924f73].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/94d97f21-9567-4a5e-a67a-2615db924f73?project=227224402169].
 REMOTE BUILD OUTPUT 
starting build "94d97f21-9567-4a5e-a67a-2615db924f73"

FETCHSOURCE
Fetching storage object: gs://cloudtutorial-279003_cloudbuild/source/1593505180.657975-20d6920b28dd4e759ca67cf216c96337.tgz#1593505180939192
Copying gs://cloudtutorial-279003_cloudbuild/source/1593505180.657975-20d6920b28dd4e759ca67cf216c96337.tgz#1593505180939192...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation completed over 1 objects/1.4 KiB.                                      
BUILD
Al

### Deploying with GCE (to use Cloud Run please)

When we deploy our image into container in **GCE**, we need to use **Docker** command to deploy our image, but I found currently with notebook, I couldn't connect with **GCE**, but with console, we could just connect with **SSH**. 

So maybe to deploy docker image in GCP, **Cloud Run** is also a good choice.

In [ ]:
# let's try to run the docker image, we need to create VM with GCE
# here I just use command to create： create-with-container
# As currently should just update the container:  update-container
! gcloud compute instances update-container first-docker --container-image gcr.io/cloudtutorial-279003/flask-container

No zone specified. Using zone [asia-east1-a] for instance: [first-docker].


#### Connect to GCE

In [ ]:
# if we want to connect to GCE, we need to provide with credentials
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = [x for x in os.listdir('/content') if x.lower().startswith('cloud')][0]

In [ ]:
# let's check with the start log
! gcloud compute instances get-serial-port-output first-docker 

#### Check images

In [ ]:
# let's check what images we have
! gcloud container images list

NAME
gcr.io/cloudtutorial-279003/flask-container
gcr.io/cloudtutorial-279003/quickstart-image
Only listing images in gcr.io/cloudtutorial-279003. Use --repository to list images in other repositories.


#### **Deploy with cloud run(recommended)**



In [220]:
# let's deploy the container to cloud run
! gcloud run deploy flask-docker --image gcr.io/cloudtutorial-279003/flask-container:latest --allow-unauthenticated --platform managed --region asia-east1

Deploying container to Cloud Run service [flask-docker] in project [cloudtutorial-279003] region [asia-east1]
Service [flask-docker] revision [flask-docker-00001-mip] has been deployed and is serving 100 percent of traffic at https://flask-docker-7so5lpt7fq-de.a.run.app


#### Check result with Cloud Run

let's check with our deployed image with the link from link up: https://flask-docker-7so5lpt7fq-de.a.run.app/ . We would get the result `Hello, kubernetes!`. 

So great! We could deploy our image into **Cloud Run**, one thing to notice is that with the use of cloud run, this is fully-managed service, if we get more traffic, it will auto-scale to start more containers, otherwise will get shutdown some containers.

In [221]:
# let's check the current service
! gcloud run services list --platform managed

   SERVICE       REGION      URL                                           LAST DEPLOYED BY        LAST DEPLOYED AT
✔  flask-docker  asia-east1  https://flask-docker-7so5lpt7fq-de.a.run.app  gqianglu1990@gmail.com  2020-06-30T08:20:43.496Z


In [222]:
# get more info
! gcloud run services describe flask-docker --platform managed --region asia-east1

✔ Service flask-docker in region asia-east1
 
Traffic: https://flask-docker-7so5lpt7fq-de.a.run.app
  100% LATEST (currently flask-docker-00001-mip)
 
Last updated on 2020-06-30T08:20:43.496Z by gqianglu1990@gmail.com:
  Revision flask-docker-00001-mip
  Image:         gcr.io/cloudtutorial-279003/flask-container:latest
  Port:          8080
  Memory:        256Mi
  CPU:           1000m
  Concurrency:   80
  Max Instances: 1000
  Timeout:       300s


In [223]:
# let's remove the service
! gcloud run services delete flask-docker --platform managed --region asia-east1

Service [flask-docker] will be deleted.

Do you want to continue (Y/n)?  y

Deleted service [flask-docker].


### **Kubernetes in action**

As above is to use **Docker** to deploy the image, but if we want to scale our application, we should use **Kubernetes** to scale and heal our application from failure.

Let's just create a sample k8s cluster for our use case and deploy our flask application in cloud.

#### yaml file

When we want to deploy our application into **GKE**, we should first define our **YAML** file to describe info about our application.

This is our image: `gcr.io/cloudtutorial-279003/flask-container:latest`.

In [127]:
# if we want to connect to GCE, we need to provide with credentials
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = [x for x in os.listdir('/content') if x.lower().startswith('cloud')][0]

#### Create cluster with command line

In [107]:
# before we do anything, let's first create a GKE cluster
# this command line is from the GKE console **command line** result to create the cluster
! gcloud beta container --project "cloudtutorial-279003" clusters create "cluster-1" --zone "us-central1-c" --no-enable-basic-auth --cluster-version "1.14.10-gke.36" --machine-type "n1-standard-1" --image-type "COS" --disk-type "pd-standard" --disk-size "100" --metadata disable-legacy-endpoints=true --scopes "https://www.googleapis.com/auth/devstorage.read_only","https://www.googleapis.com/auth/logging.write","https://www.googleapis.com/auth/monitoring","https://www.googleapis.com/auth/servicecontrol","https://www.googleapis.com/auth/service.management.readonly","https://www.googleapis.com/auth/trace.append" --num-nodes "3" --enable-stackdriver-kubernetes --enable-ip-alias --network "projects/cloudtutorial-279003/global/networks/default" --subnetwork "projects/cloudtutorial-279003/regions/us-central1/subnetworks/default" --default-max-pods-per-node "110" --no-enable-master-authorized-networks --addons HorizontalPodAutoscaling,HttpLoadBalancing --enable-autoupgrade --enable-autorepair --max-surge-upgrade 1 --max-unavailable-upgrade 0

client [kubectl]. To install, run
  $ gcloud components install kubectl

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
ERROR: (gcloud.beta.container.clusters.create) ResponseError: code=409, message=Already exists: projects/cloudtutorial-279003/zones/us-central1-c/clusters/cluster-1.


In [109]:
# so that we have created a cluster, then we need to config the crediential of the cluster
! gcloud container clusters get-credentials cluster-1 --zone us-central1-c

client [kubectl]. To install, run
  $ gcloud components install kubectl

Fetching cluster endpoint and auth data.
kubeconfig entry generated for cluster-1.


#### install cloud SDK for kubectl

In [119]:
# then let's install the kubectl to interact with k8s
! echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
! sudo apt-get install apt-transport-https ca-certificates gnupg


deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20190110~18.04.1).
gnupg is already the newest version (2.2.4-1ubuntu1.2).
apt-transport-https is already the newest version (1.6.12ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [120]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key --keyring /usr/share/keyrings/cloud.google.gpg add -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  25115      0 --:--:-- --:--:-- --:--:-- 25115
OK


In [ ]:
# install gcloud-sdk
! sudo apt-get update && sudo apt-get install google-cloud-sdk

In [ ]:
# install kubectl
! sudo apt-get install kubectl

#### Check Kubectl

In [130]:
# we have to config the GOOGLE_APPLICATION_CREDENTIALS with the crediential file.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/CloudTutorial-e8d69ceedc4d.json"

In [132]:
# let's check we do install the kubectl API.
! kubectl version

Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.4", GitCommit:"c96aede7b5205121079932896c4ad89bb93260af", GitTreeState:"clean", BuildDate:"2020-06-17T11:41:22Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"14+", GitVersion:"v1.14.10-gke.36", GitCommit:"34a615f32e9a0c9e97cdb9f749adb392758349a6", GitTreeState:"clean", BuildDate:"2020-04-06T16:33:17Z", GoVersion:"go1.12.12b4", Compiler:"gc", Platform:"linux/amd64"}


#### Pods defination

In [186]:
%%writefile config.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app.kubernetes.io/name: flask-container
  name: flask-container
spec:
  replicas: 5
  selector:
    matchLabels:
      app.kubernetes.io/name: flask-container
  template:
    metadata:
      labels:
        app.kubernetes.io/name: flask-container
    spec:
      containers:
      - image: gcr.io/cloudtutorial-279003/flask-container:latest
        name: hello-flask
        ports:
        - containerPort: 8080

Overwriting config.yaml


In [ ]:
# first try to delete the deployment
! kubectl delete deployments flask-container

#### Deploy the cluster with pods

In [229]:
# let's apply with the config 
! kubectl apply -f config.yaml

deployment.apps/flask-container created


In [153]:
# after we have already created the cluster, let's deploy our image into the cluster
# this is to create a deployment first.
# ! kubectl create deployment flask-container --image=gcr.io/cloudtutorial-279003/flask-container:latest 

Error from server (AlreadyExists): deployments.apps "flask-container" already exists


In [230]:
# let's check the deplyment
! kubectl get deployments flask-container

NAME              READY   UP-TO-DATE   AVAILABLE   AGE
flask-container   3/5     5            3           4s


In [191]:
# get more info about the deployment
! kubectl describe deployments flask-container

Name:                   flask-container
Namespace:              default
CreationTimestamp:      Tue, 30 Jun 2020 06:57:36 +0000
Labels:                 app.kubernetes.io/name=flask-container
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app.kubernetes.io/name=flask-container
Replicas:               5 desired | 5 updated | 5 total | 5 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app.kubernetes.io/name=flask-container
  Containers:
   hello-flask:
    Image:        gcr.io/cloudtutorial-279003/flask-container:latest
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSet

#### Replicasets

In [192]:
# let's get some info about replicates
! kubectl get replicasets

NAME                         DESIRED   CURRENT   READY   AGE
flask-container-6747ff5cf5   5         5         5       68m


In [167]:
# describe replicasets
! kubectl describe replicasets

Name:           flask-container-6747ff5cf5
Namespace:      default
Selector:       app.kubernetes.io/name=flask-container,pod-template-hash=6747ff5cf5
Labels:         app.kubernetes.io/name=flask-container
                pod-template-hash=6747ff5cf5
Annotations:    deployment.kubernetes.io/desired-replicas: 5
                deployment.kubernetes.io/max-replicas: 7
                deployment.kubernetes.io/revision: 1
Controlled By:  Deployment/flask-container
Replicas:       5 current / 5 desired
Pods Status:    5 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  app.kubernetes.io/name=flask-container
           pod-template-hash=6747ff5cf5
  Containers:
   hello-flask:
    Image:        gcr.io/cloudtutorial-279003/flask-container:latest
    Port:         8080/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From                   Message
  ----    ------            ----

#### Pods info

In [231]:
# before we do anything, we should check each pod status
# so that we do get each pod `running`
! kubectl get pods --output=wide

NAME                               READY   STATUS    RESTARTS   AGE   IP          NODE                                       NOMINATED NODE   READINESS GATES
flask-container-6747ff5cf5-fcczd   1/1     Running   0          14s   10.4.1.9    gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-kvv6n   1/1     Running   0          14s   10.4.1.10   gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-lhmpl   1/1     Running   0          14s   10.4.0.11   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-rvznh   1/1     Running   0          14s   10.4.2.6    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>
flask-container-6747ff5cf5-v74zv   1/1     Running   0          14s   10.4.2.5    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>


In [234]:
# delete service
! kubectl delete service my-first-service

service "my-first-service" deleted


#### Expose service to worldwide

In [235]:
# let's create service, so that we could get response by out ip
! kubectl expose deployment flask-container --type=LoadBalancer --name=my-first-service

service/my-first-service exposed


In [239]:
# let's check our service, if we get the external_ip, then we could visit that IP
! kubectl get service my-first-service

NAME               TYPE           CLUSTER-IP   EXTERNAL-IP    PORT(S)          AGE
my-first-service   LoadBalancer   10.8.2.181   34.71.78.151   8080:32365/TCP   90s


In [203]:
# get more detail info about my service
! kubectl describe service my-first-service

Name:                     my-first-service
Namespace:                default
Labels:                   app.kubernetes.io/name=flask-container
Annotations:              <none>
Selector:                 app.kubernetes.io/name=flask-container
Type:                     LoadBalancer
IP:                       10.8.14.76
LoadBalancer Ingress:     104.154.68.183
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  31906/TCP
Endpoints:                10.4.0.10:8080,10.4.1.6:8080,10.4.1.7:8080 + 2 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason                Age    From                Message
  ----    ------                ----   ----                -------
  Normal  EnsuringLoadBalancer  2m13s  service-controller  Ensuring load balancer
  Normal  EnsuredLoadBalancer   95s    service-controller  Ensured load balancer


In [214]:
# as I always get connection-refused, so try to create firewall rules. not ok...
! gcloud compute firewall-rules create my-rule-30809  --allow=tcp:30809 

Created [https://www.googleapis.com/compute/v1/projects/cloudtutorial-279003/global/firewalls/my-rule-30809].
NAME           NETWORK  DIRECTION  PRIORITY  ALLOW      DENY  DISABLED
my-rule-30809  default  INGRESS    1000      tcp:30809        False


#### Connect with service

In [240]:
# let's visit our website IP with curl, 
# we do get response with: `Hello, kubernetes!`
! curl http://34.71.78.151:8080

Hello, kubernetes!

In [241]:
# last let's just check the pods status
! kubectl get pods --output=wide

NAME                               READY   STATUS    RESTARTS   AGE     IP          NODE                                       NOMINATED NODE   READINESS GATES
flask-container-6747ff5cf5-fcczd   1/1     Running   0          4m36s   10.4.1.9    gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-kvv6n   1/1     Running   0          4m36s   10.4.1.10   gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-lhmpl   1/1     Running   0          4m36s   10.4.0.11   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-rvznh   1/1     Running   0          4m36s   10.4.2.6    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>
flask-container-6747ff5cf5-v74zv   1/1     Running   0          4m36s   10.4.2.5    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>


### Scale with K8S

In [247]:
! kubectl get replicasets

NAME                         DESIRED   CURRENT   READY   AGE
flask-container-6747ff5cf5   5         5         5       8m5s


In [250]:
# we could just replicate the 5 to 10
! kubectl scale  --replicas=10 deployment/flask-container

deployment.extensions/flask-container scaled


In [251]:
# let's check current replicas
! kubectl get replicasets

NAME                         DESIRED   CURRENT   READY   AGE
flask-container-6747ff5cf5   10        10        10      9m12s


In [252]:
# check pods
# so we have scaled our application from 5 to 10
! kubectl get pods --output=wide

NAME                               READY   STATUS    RESTARTS   AGE     IP          NODE                                       NOMINATED NODE   READINESS GATES
flask-container-6747ff5cf5-2kssm   1/1     Running   0          42s     10.4.0.13   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-fcczd   1/1     Running   0          9m31s   10.4.1.9    gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-kvv6n   1/1     Running   0          9m31s   10.4.1.10   gke-cluster-1-default-pool-a635d57a-4pbm   <none>           <none>
flask-container-6747ff5cf5-lhmpl   1/1     Running   0          9m31s   10.4.0.11   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-mm4pf   1/1     Running   0          42s     10.4.0.12   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-pwwjc   1/1     Running   0          42s     10.4.2.8    g

In [254]:
# scale-down
! kubectl scale --replicas=3 deployment/flask-container

deployment.extensions/flask-container scaled


In [255]:
# let's check current replicas
! kubectl get replicasets

NAME                         DESIRED   CURRENT   READY   AGE
flask-container-6747ff5cf5   3         3         3       10m


In [256]:
# check pods
# so we have scaled our application from 5 to 10
! kubectl get pods --output=wide

NAME                               READY   STATUS    RESTARTS   AGE   IP          NODE                                       NOMINATED NODE   READINESS GATES
flask-container-6747ff5cf5-lhmpl   1/1     Running   0          10m   10.4.0.11   gke-cluster-1-default-pool-a635d57a-70ql   <none>           <none>
flask-container-6747ff5cf5-rvznh   1/1     Running   0          10m   10.4.2.6    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>
flask-container-6747ff5cf5-v74zv   1/1     Running   0          10m   10.4.2.5    gke-cluster-1-default-pool-a635d57a-shb8   <none>           <none>


In [257]:
import requests

url = "http://34.71.78.151:8080"

response = requests.get(url)

# we just get same result with even 3 pods....
# that's the magic of K8S
print(response.text)

Hello, kubernetes!


#### Clean up

We have seen that we do could create a service with **K8S** cluster to expose worldwide that we could get response to that service, and we could also scale up or down with that service.

Last we should try to just delete our cluster and service we created.

In [258]:
! kubectl delete service my-first-service

service "my-first-service" deleted


In [259]:
# delete deployment
! kubectl delete deployment flask-container

deployment.extensions "flask-container" deleted


In [261]:
# delete cluster
! gcloud container clusters delete cluster-1 --zone us-central1-c

The following clusters will be deleted.
 - [cluster-1] in [us-central1-c]

Do you want to continue (Y/n)?  y

Deleted [https://container.googleapis.com/v1/projects/cloudtutorial-279003/zones/us-central1-c/clusters/cluster-1].
